# Getting Chr5 novel node statistics

This is more or less following the hpgp-basics doc, but through python rather than R.

First, we need to import stuff:

In [100]:
import os
import subprocess
import pandas as pd
import seaborn as sns

node_dir = "nonref_files/chr5-1k-10-50/"
nodes = os.listdir(node_dir)

sample_data = pd.read_csv("nonref_files/20130606_g1k_3202_samples_ped_population.txt", 
                         delimiter=" ")
sample_data["SampleID"] = sample_data["SampleID"].astype(str)

sample_data
print(nodes[:3])

['4921090.txt', '4905845.txt', '4950687.txt']


Now we need to get all the path information for each node, and convert it into a dataframe so we can do something useful with it.

In [101]:
pop_counts = {}
node_data = ""
nid = ""

for node in nodes[:1]: # only working on the first node for now
    nid = node[:node.find(".")]
    paths = open(node_dir + node).read().split('\n')[:-2] # 2 trailing newlines
    
    counts = []
    # converting to a set makes sure homozygotes are only counted once
    for path in sorted(list(set(paths))):
        counts.append((path, paths.count(path)))
    
    # grab population counts
    test = pd.DataFrame(counts, columns=["SampleID", "count"])
    test["SampleID"] = test["SampleID"].astype(str)
    test = test.merge(sample_data[["SampleID", "Population"]], on="SampleID")
    pop_counts[nid] = test["Population"].value_counts()
    
    # we can work on the whole thing later, but let's just grab one node for now.
    node_data = test
    print(node_data)

# some completely unnecessary data transformation - may be interesting later...    
test = pd.DataFrame.from_dict(pop_counts).reset_index()
test = pd.pivot_table(test, columns="Population")

   SampleID  count Population
0   HG01071      1        PUR
1   HG01109      1        PUR
2   HG01175      1        PUR
3   HG01891      1        ACB
4   HG02080      1        KHV
5   HG02145      1        ACB
6   HG02148      1        PEL
7   HG02622      2        GWD
8   HG02630      1        GWD
9   HG02723      1        GWD
10  HG02886      2        GWD
11  HG03098      1        MSL
12  HG03486      1        MSL
13  HG03492      1        PJL
14  HG03579      1        MSL


We can now use `node_data` to get information about the populations in our node.

In [102]:
print(f"Counts for {nid}:")
print(node_data["Population"].value_counts())

Counts for 4921090:
Population
GWD    4
PUR    3
MSL    3
ACB    2
KHV    1
PEL    1
PJL    1
Name: count, dtype: int64


We can see that most of the paths represented are part of the GWD, PUR, and MSL populations.

We can then grab the path prefixes 

In [117]:
populations = pd.read_csv("data/20131219.populations.tsv", delimiter="\t")
print(populations[populations["Population Code"] == "GWD"]["Population Description"])



print(node_data[node_data["Population"]=="GWD"])

path_prefixes = node_data[node_data["Population"]=="GWD"]["SampleID"].tolist()

13    Gambian in Western Division, The Gambia
Name: Population Description, dtype: object
   SampleID  count Population
7   HG02622      2        GWD
8   HG02630      1        GWD
9   HG02723      1        GWD
10  HG02886      2        GWD


Due to the way we extracted the path names, we have to rerun odgi.

Because I am silly, we have to extract, then get the paths, *then* extract again. Also note that this also grabs heterozygotes (we can deal with that later).

We can also use the interface (or just odgi viz) to grab the visualisation for this node - or visualise the wider region (probably better).

In [120]:
node_og = f"nonref_files/chr5-1k-10-50/node_{nid}.og"

subprocess.run(["./bin/odgi", "extract", "-i", "chr5.full.og", "-n", nid,
                "-o", node_og, "-t", "4"]);

In [123]:
paths = subprocess.run(["./bin/odgi", "paths", "-i", node_og,
                       "-L"], capture_output=True).stdout.decode().split()
        
paths = [p for p in paths if p[:p.find("#")] in path_prefixes]
print(paths)

f = open("_paths.txt", "w") #remove later

for p in paths:
    f.write(p + "\n")

f.close()

['HG02622#1#JAHAOO010000060.1#0:27547620-27548750', 'HG02622#2#JAHAON010000051.1#0:21697577-21698707', 'HG02630#2#JAHAOP010000044.1#0:21714757-21715887', 'HG02723#2#JAHEOT010000033.1#0:26930289-26931419', 'HG02886#1#JAHAOU010000047.1#0:21754255-21755385', 'HG02886#2#JAHAOT010000044.1#0:21627707-21628837']


In theory, we could then extract specific paths - no need to do that here; paths in the graph are all exactly the same within that node. (See visualisation in the `img` folder of the repo, alternatively you can run `odgi viz` to get sequence info).